In [1]:
#
# need itertools to run the code below
#
import itertools

In [2]:
#
# icosahedron = dual simplicial complex to the dodecahedron
#
Icosahedron = SimplicialComplex([[0,1,2],
                                 [0,1,3],
                                 [0,2,4],
                                 [1,2,6],
                                 [0,3,5],
                                 [0,4,5],
                                 [1,3,7],
                                 [1,6,7],
                                 [2,4,8],
                                 [2,6,8],
                                 [4,5,10],
                                 [4,8,10],
                                 [3,5,9],
                                 [3,7,9],
                                 [6,7,11],
                                 [6,8,11],
                                 [5,9,10],
                                 [7,9,11],
                                 [8,10,11],
                                 [9,10,11]])
#
# 1-skeleton of the icosahedron
#
Skeleton = Icosahedron.graph()

In [3]:
#
# all possible ZZ^4_2 colours
#
Colours = [(1,0,0,0), #0
           (0,1,0,0), #1
           (0,0,1,0), #2
           (0,0,0,1), #3
           (1,1,0,0), #4
           (1,0,1,0), #5
           (1,0,0,1), #6
           (0,1,1,0), #7
           (0,1,0,1), #8
           (0,0,1,1), #9
           (0,1,1,1), #10
           (1,0,1,1), #11
           (1,1,0,1), #12
           (1,1,1,0), #13
           (1,1,1,1)] #14
Colours = [vector(GF(2), v) for v in Colours]
#
# the base colouring to extend: the only orientable small cover of the right-angled
# dodecahedron found by Garrison and Scott in "Small Covers of the Dodecahedron and 
# the 120-cell" that is QHS
#
Base = [[1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
        [0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1],
        [0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0]]
#
# defining matrix of the base colouring
#
L0 = matrix(GF(2), Base)
#
# itw row space
#
Row = L0.row_space().list()
#
# cardinality of the row space
#
num_vectors = len(Row) 
#
# dimension of the simplicial complex = 2
#
n = dimension(Icosahedron)
#
# number of vertices of the icosahedron = dual simplicial complex to the dodecahedron
#
num_vertices = 12 
#
# the row vector space for the defining matrices of colourings
#
V = VectorSpace(GF(2), num_vertices).list()

In [4]:
#
# looking for possible QHS extensions
#
QHS_extensions = []
for v in V:
    if not(v in Row) and (tuple(v[:3])==(0,0,0)):
        count = 0
        for vector in Row:
            test = []
            for i in range(num_vertices):
                if v[i]+vector[i]==1:
                    test.append(i)
            if not(Skeleton.subgraph(test).is_connected()):
                break
            else:
                count = count + 1
        if count==num_vectors:
            L = matrix(GF(2),Base+[v])
            colouring = []
            for i in range(num_vertices):
                colour = L.column(i)
                colouring.append(Colours.index(colour))
            QHS_extensions.append(colouring)
# How many possible QHS extensions in total? (should be 53)
print('Found {} QHS extensions'.format(len(QHS_extensions)))

Found 53 QHS extensions


In [5]:
#
# the group of symmetries of the regular dodecahedron / icosahedron
#
Gamma = Icosahedron.automorphism_group()
#
# colourings DJ-equivalent to v under the action of symmetry g 
# (if exist, othervise returns an empty list)
#
def DJ_equivalence(g, v):
    num_vertices = 12   # number of vertices of the icosahedron 
    num_colours = 15    # number of colours used = 2^4 - 1
    equiv = []
    for col in Colours:
        L0 = matrix(GF(2), [Colours[v[g(0)]], Colours[v[g(1)]], Colours[v[g(2)]], col]).transpose()
        if L0.det()!=0:
            L = L0.inverse()
            lst = []
            for i in range(num_vertices):
                vec = L*Colours[v[g(i)]]
                j = Colours.index(vec)
                lst.append(j)
            equiv.append(lst)
    return equiv

In [6]:
#
# order of the symmetry group
#
sym_order = Gamma.order()
#
# finding representatives of DJ-equivalence classes among all possible QHS extensions
# obtained before: here we need 1 representative of each equivalence class only
#
QHS_extension_classes = []
for v in QHS_extensions:
    count_equiv = 0
    for g in Gamma:
        equiv_g = DJ_equivalence(g, v)
        num_equiv_g = len(equiv_g)
        count_g = 0
        for colouring in equiv_g:
            if (colouring!=v) and (colouring in QHS_extension_classes):
                break
            else:
                count_g = count_g + 1
        if count_g==num_equiv_g:
            count_equiv = count_equiv + 1
    if count_equiv==sym_order:
        QHS_extension_classes.append(v)
# How many DJ-equivalence classes? (should be 7)
print('Among them {} DJ-equivalence classes'.format(len(QHS_extension_classes)))

Among them 7 DJ-equivalence classes


In [7]:
#
# looking for QHS with non-trivial coloured symmetries 
# in the list obtained above: there should be none
#
Sym_QHS = []
for v in QHS_extension_classes:
    sym = []
    for g in Gamma:
        equiv = DJ_equivalence(g, v)
        for colouring in equiv:
            if colouring==v:
                sym.append(g)
    Sym_QHS.append((v, Gamma.subgroup(sym)))

In [8]:
#
# printing out the DJ-equivalence classes of colouring extensions
#
count = 0
for (v, H) in Sym_QHS:
    count = count+1
    print("Colouring {}".format(count))
    print(v)
    print("Defining matrix:")
    Lambda = matrix([Colours[i] for i in v]).transpose()
    print(Lambda)
    print("-"*25)
    struct = H.structure_description()
    print("Coloured symmetries: {}".format(struct))
    print("%"*25)

Colouring 1
[0, 1, 2, 9, 8, 14, 6, 14, 13, 0, 2, 1]
Defining matrix:
[1 0 0 0 0 1 1 1 1 1 0 0]
[0 1 0 0 1 1 0 1 1 0 0 1]
[0 0 1 1 0 1 0 1 1 0 1 0]
[0 0 0 1 1 1 1 1 0 0 0 0]
-------------------------
Coloured symmetries: 1
%%%%%%%%%%%%%%%%%%%%%%%%%
Colouring 2
[0, 1, 2, 9, 1, 14, 0, 13, 13, 6, 2, 1]
Defining matrix:
[1 0 0 0 0 1 1 1 1 1 0 0]
[0 1 0 0 1 1 0 1 1 0 0 1]
[0 0 1 1 0 1 0 1 1 0 1 0]
[0 0 0 1 0 1 0 0 0 1 0 0]
-------------------------
Coloured symmetries: C3
%%%%%%%%%%%%%%%%%%%%%%%%%
Colouring 3
[0, 1, 2, 2, 8, 14, 0, 13, 13, 6, 2, 1]
Defining matrix:
[1 0 0 0 0 1 1 1 1 1 0 0]
[0 1 0 0 1 1 0 1 1 0 0 1]
[0 0 1 1 0 1 0 1 1 0 1 0]
[0 0 0 0 1 1 0 0 0 1 0 0]
-------------------------
Coloured symmetries: 1
%%%%%%%%%%%%%%%%%%%%%%%%%
Colouring 4
[0, 1, 2, 9, 8, 14, 0, 14, 13, 6, 2, 1]
Defining matrix:
[1 0 0 0 0 1 1 1 1 1 0 0]
[0 1 0 0 1 1 0 1 1 0 0 1]
[0 0 1 1 0 1 0 1 1 0 1 0]
[0 0 0 1 1 1 0 1 0 1 0 0]
-------------------------
Coloured symmetries: C2
%%%%%%%%%%%%%%%%%%%%%%%%%
Colour

In [9]:
#
# colouring extension M from the proof of Theorem 5.5
#
m0 = [0, 1, 9, 2, 1, 13, 6, 13, 14, 0, 2, 1]
M0 = matrix(Colours[i] for i in m0).transpose()
print("The initial defining matrix:")
print(M0)
#
# we need an equivalent colouring with the first three 
# colours being the standard basis vectors e_1, e_2, e_3
# since all DJ-equivalence classes have such "reduced"
# up to the action of GL_4(Z_2) representatives: this 
# approach allows us to compute more efficiently 
# 
C = [[1,0,0,0],
     [0,1,0,0],
     [0,0,1,0],
     [0,0,1,1]]
C = matrix(GF(2), C)
print("We need an equivalent colouring starting with e_1, e_2, e_3 standard basis vectors ...")
print("Its defining matrix:")
M = C*M0
print(M)
m = [Colours.index(v) for v in M.transpose()]

The initial defining matrix:
[1 0 0 0 0 1 1 1 1 1 0 0]
[0 1 0 0 1 1 0 1 1 0 0 1]
[0 0 1 1 0 1 0 1 1 0 1 0]
[0 0 1 0 0 0 1 0 1 0 0 0]
We need an equivalent colouring starting with e_1, e_2, e_3 standard basis vectors ...
Its defining matrix:
[1 0 0 0 0 1 1 1 1 1 0 0]
[0 1 0 0 1 1 0 1 1 0 0 1]
[0 0 1 1 0 1 0 1 1 0 1 0]
[0 0 0 1 0 1 1 1 0 0 1 0]


In [10]:
#
# which DJ-equivalence class does M belong to?
#
equiv = []
for (v, H) in Sym_QHS:
    for g in Gamma:
        equiv_g = DJ_equivalence(g, v)
        if m in equiv_g:
            equiv += [(v, H)]
#
# sanity check: the list "equiv" that we found should contain several copies 
# of the _same_ entry (in this case 3 copies)
#
print(len(equiv)==3)
#
print(equiv[0]==equiv[1]==equiv[2])

True
True


In [11]:
#
# the equivalence class of M has index 5 in the list of all 
# colouring classes (and their coloured symmetyries) Sym_QHS
#
equiv[0]==Sym_QHS[5]

True